In [1]:
import pandas as pd
import numpy as np
import random
import math

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import scipy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
from tabm_reference import Model, make_parameter_groups
import rtdl_num_embeddings
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

from LG_Aimers_6th.cal_auc import calculate_auc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## 2. Data Load

In [2]:
data_seed = 1

train_path = f'../data/custom_train_{data_seed}.csv'
test_path = f'../data/custom_test_{data_seed}.csv'

# 학습/평가 데이터 로드
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

print(train.shape, test.shape)

(205080, 68) (51271, 67)


In [3]:
from preprocess_DL import all_process

train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

train, test = all_process(train, test)

cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 여부']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

수치형 변수: 57개 
['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '시술_임신', '배아생성이유_기증용', '배아생성이유_난자 저장용', '배아생성이유_배아 저장용', '배아생성이유_현재 시술용']
범주형 변수: 8개 
['시술 시기 코드', '시술 당시 나이', '배란 유도 유형', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(205080, 66) (51271, 65)


In [4]:
def df_to_tensor(train, val, test=None):
    if test is None:
        data_numpy = {
            'train': {'x_cat': train[cat_cols].values, 'x_cont': train[numeric_cols].values, 'y': train['임신 성공 여부'].values},
            'val': {'x_cat': val[cat_cols].values, 'x_cont': val[numeric_cols].values, 'y': val['임신 성공 여부'].values},
        }
        data = {
            part: {k: torch.as_tensor(v, device=device) for k, v in data_numpy[part].items()}
            for part in data_numpy
        }
        data['train']['y'] = data['train']['y'].long()
        data['val']['y'] = data['val']['y'].long()
        return data
    else:
        data_numpy = {
            'train': {'x_cat': train[cat_cols].values, 'x_cont': train[numeric_cols].values, 'y': train['임신 성공 여부'].values},
            'val': {'x_cat': val[cat_cols].values, 'x_cont': val[numeric_cols].values, 'y': val['임신 성공 여부'].values},
            'test': {'x_cat': test[cat_cols].values, 'x_cont': test[numeric_cols].values},
        }
        data = {
            part: {k: torch.as_tensor(v, device=device) for k, v in data_numpy[part].items()}
            for part in data_numpy
        }
        data['train']['y'] = data['train']['y'].long()
        data['val']['y'] = data['val']['y'].long()
        return data


In [5]:
def get_feature_info(train):
    n_num_features_ = len(numeric_cols)
    cat_cardinalities_ = [train[col].nunique() for col in cat_cols]

    return n_num_features_, cat_cardinalities_

n_num_features, cat_cardinalities = get_feature_info(train)
bins = rtdl_num_embeddings.compute_bins(torch.tensor(train[numeric_cols].values))

model_config = {
    'n_num_features': n_num_features,
    'cat_cardinalities': cat_cardinalities,
    'n_classes': 2,
    'backbone': {
        'type': 'MLP',
        'n_blocks': 3 if bins is None else 2,
        'd_block': 512,
        'dropout': 0.1,
    },
    'bins': bins,
    'num_embeddings': (
        None if bins is None else {
            'type': 'PiecewiseLinearEmbeddings',
            'd_embedding': 16,
            'activation': False,
            'version': 'B',
        }
    ),
    'arch_type': 'tabm-mini',
    'k': 32,
    'share_training_batches': True,
}

model = Model(**model_config).to(device)

print(n_num_features)
print(cat_cardinalities)

57
[7, 7, 2, 3, 4, 5, 7, 9]


In [6]:
optimizer = torch.optim.AdamW(make_parameter_groups(model), lr=2e-3, weight_decay=3e-4)

# 기본 손실 함수 설정 (여기서는 분류 문제이므로 cross_entropy)
base_loss_fn = F.cross_entropy

def loss_fn(y_pred, y_true):
    # 모델 출력 y_pred.shape: (batch_size, k, n_classes)
    k = y_pred.shape[-2]  # k 예측 개수
    # 모든 예측에 대해 손실을 계산
    return base_loss_fn(
        y_pred.flatten(0, 1),
        y_true.repeat_interleave(k) if model.share_training_batches else y_true
    )

def apply_model(part: str, idx: Tensor) -> Tensor:
    return (
        model(
            data[part]['x_cont'][idx],
            data[part]['x_cat'][idx] if 'x_cat' in data[part] else None,
        )
        .squeeze(-1)  # Remove the last dimension for regression tasks.
        .float()
    )

def evaluate(part: str, metric: str = 'auc') -> float:
    model.eval()
    eval_batch_size = 2048

    all_preds = []
    all_targets = []
    with torch.no_grad():
        for idx in torch.arange(len(data[part]['y']), device=device).split(eval_batch_size):
            preds = apply_model(part, idx)
            all_preds.append(preds)
            all_targets.append(data[part]['y'][idx])
    # 예측 결과: (N, k, n_classes) -> (N, n_classes) (k개의 예측의 평균)
    y_pred = torch.cat(all_preds).cpu().numpy()
    y_true = torch.cat(all_targets).cpu().numpy()

    # 소프트맥스를 통해 확률로 변환한 뒤 k개 예측에 대해 평균을 냅니다.
    y_pred = scipy.special.softmax(y_pred, axis=-1)
    y_pred = y_pred.mean(1)  # shape: (N, n_classes)

    if metric == 'auc':
        auc = roc_auc_score(y_true, y_pred[:, 1])
        return float(auc)
    elif metric == 'loss':
        total_loss = 0.0
        total_samples = 0
        with torch.no_grad():
            for idx in torch.arange(len(data[part]['y']), device=device).split(eval_batch_size):
                preds = apply_model(part, idx)
                batch_loss = loss_fn(preds, data[part]['y'][idx])
                total_loss += batch_loss.item() * idx.size(0)
                total_samples += idx.size(0)
        return total_loss / total_samples
    else:
        raise ValueError(f"Unknown metric: {metric}")


def predict_proba(part: str) -> np.ndarray:
    model.eval()
    eval_batch_size = 2048
    preds = []
    with torch.no_grad():
        num_samples = len(data[part]['x_cont'])
        for idx in torch.arange(num_samples, device=device).split(eval_batch_size):
            batch_pred = apply_model(part, idx)
            preds.append(batch_pred)

    # 모든 배치의 예측값을 하나의 텐서로 합칩니다.
    y_pred = torch.cat(preds).cpu().numpy()
    y_pred = scipy.special.softmax(y_pred, axis=-1)

    return y_pred.mean(1)

In [7]:
# 1. 자동 혼합 정밀도(AMP)에서 사용할 데이터 타입(amp_dtype)을 설정합니다.
if torch.cuda.is_available():
    # CUDA를 사용할 수 있으면, bfloat16 지원 여부를 확인합니다.
    if torch.cuda.is_bf16_supported():
        amp_dtype = torch.bfloat16  # bfloat16을 지원하면 bfloat16을 사용합니다.
    else:
        amp_dtype = torch.float16   # bfloat16을 지원하지 않으면 float16을 사용합니다.
else:
    amp_dtype = None  # CUDA를 사용할 수 없으면 AMP를 사용하지 않습니다.

# 2. AMP 기능을 활성화할지 결정합니다.
amp_enabled = False and (amp_dtype is not None)

# 3. GradScaler는 float16 모드에서 수치 안정성을 위해 사용됩니다.
# amp_dtype이 torch.float16일 때만 GradScaler를 생성합니다.
if amp_dtype == torch.float16:
    grad_scaler = torch.cuda.amp.GradScaler()
else:
    grad_scaler = None

print(grad_scaler)

None


In [8]:
seed = 333

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

test_preds = []
val_scores = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train['임신 성공 여부'])):
    fold_train = train.iloc[train_idx].copy().reset_index(drop=True)
    fold_valid = train.iloc[valid_idx].copy().reset_index(drop=True)
    fold_train2 = fold_train.copy()
    fold_test = test.copy()

    fold_train, fold_valid = all_process(fold_train, fold_valid)
    _, fold_test = all_process(fold_train2, fold_test)

    cat_cols = [col for col in fold_train.columns if pd.api.types.is_categorical_dtype(fold_train[col])]
    numeric_cols = [col for col in fold_train.columns if col not in cat_cols and col != '임신 성공 여부']

    bins = rtdl_num_embeddings.compute_bins(torch.tensor(fold_train[numeric_cols].values))
    n_num_features, cat_cardinalities = get_feature_info(fold_train)

    model_config = {
        'n_num_features': n_num_features,
        'cat_cardinalities': cat_cardinalities,
        'n_classes': 2,
        'backbone': {
            'type': 'MLP',
            'n_blocks': 3 if bins is None else 2,
            'd_block': 512,
            'dropout': 0.1,
        },
        'bins': bins,
        'num_embeddings': (
            None if bins is None else {
                'type': 'PiecewiseLinearEmbeddings',
                'd_embedding': 16,
                'activation': False,
                'version': 'B',
            }
        ),
        'arch_type': 'tabm-mini',
        'k': 32,
        'share_training_batches': True,
    }
    model = Model(**model_config).to(device)
    optimizer = torch.optim.AdamW(make_parameter_groups(model), lr=2e-3, weight_decay=3e-4)

    data = df_to_tensor(fold_train, fold_valid, fold_test)

    n_epochs = 1000000
    early_stopping = 'loss'

    train_size = len(fold_train)
    batch_size = 1024
    epoch_size = math.ceil(train_size / batch_size)
    if early_stopping == 'loss':
        best = {'val': float('inf'), 'test': float('inf'), 'epoch': -1}
    else:  # 'auc'인 경우, 값이 높을수록 좋으므로
        best = {'val': -float('inf'), 'test': -float('inf'), 'epoch': -1}

    patience = 3
    remaining_patience = patience

    print('-' * 88 + '\n')
    for epoch in range(n_epochs):
        train_loss_epoch = 0.0  # 전체 학습 손실 누적
        batches = (
            torch.randperm(train_size, device=device).split(batch_size)
            if model.share_training_batches
            else [
                x.transpose(0, 1).flatten()
                for x in torch.rand((model.k, train_size), device=device)
                .argsort(dim=1)
                .split(batch_size, dim=1)
            ]
        )

        # for batch_idx in tqdm(batches, desc=f'Epoch {epoch}'):
        for batch_idx in batches:
            model.train()
            optimizer.zero_grad()
            loss = loss_fn(apply_model('train', batch_idx), data['train']['y'][batch_idx])
            train_loss_epoch += loss.item() * batch_idx.size(0)
            if grad_scaler is None:
                loss.backward()
                optimizer.step()
            else:
                grad_scaler.scale(loss).backward()  # type: ignore
                grad_scaler.step(optimizer)
                grad_scaler.update()
        train_loss_epoch /= train_size

        if early_stopping == 'loss':
            val_metric = evaluate('val', metric='loss')
            val_auc = evaluate('val', metric='auc')
            y_test_pred = predict_proba('test')[:, 1]
            test_metric = calculate_auc(y_test_pred, data_seed)
        elif early_stopping == 'auc':
            val_metric = evaluate('val', metric='auc')
            y_test_pred = predict_proba('test')[:, 1]
            test_metric = calculate_auc(y_test_pred, data_seed)
        else:
            raise ValueError(f"Unknown metric: {early_stopping}")

        print(f"[Epoch {epoch+1}] Train Loss: {train_loss_epoch:.4f}, Val {early_stopping.capitalize()}: {val_metric:.4f},"
              f"Val AUC: {val_auc}, Test AUC: {test_metric:.4f}")

        if early_stopping == 'loss':
            if val_metric < best['val']:
                # print("🌸 New best epoch! 🌸")
                best = {'val': val_metric, 'test': test_metric, 'epoch': epoch}
                torch.save(model.state_dict(), "best_model.pt")
                remaining_patience = patience
            else:
                remaining_patience -= 1

        elif early_stopping == 'auc':
            if val_metric > best['val']:
                # print("🌸 New best epoch! 🌸")
                best = {'val': val_metric, 'test': test_metric, 'epoch': epoch}
                torch.save(model.state_dict(), "best_model.pt")
                remaining_patience = patience
            else:
                remaining_patience -= 1

        if remaining_patience < 0:
            print("Early stopping triggered.")
            break
        print()

    print('\n\nResult:')
    print(best)

    model.load_state_dict(torch.load("best_model.pt"))
    print("Best model weights loaded from best_model.pt")

    val_score = evaluate('val', metric='auc')
    val_scores.append(val_score)

    y_test_pred = predict_proba('test')[:, 1]
    test_preds.append(y_test_pred)

valid_auc = np.mean(val_scores, axis=0)

test_auc = calculate_auc(np.mean(test_preds, axis=0), data_seed)

print(f'[Data Seed {data_seed}] Validation AUC: {valid_auc:.4f}, Test AUC: {test_auc}')

----------------------------------------------------------------------------------------

[Epoch 1] Train Loss: 0.5103, Val Loss: 0.4932,Val AUC: 0.7344616417096598, Test AUC: 0.7365

[Epoch 2] Train Loss: 0.4927, Val Loss: 0.4912,Val AUC: 0.735700091756804, Test AUC: 0.7377

[Epoch 3] Train Loss: 0.4909, Val Loss: 0.4907,Val AUC: 0.7358322481111923, Test AUC: 0.7387

[Epoch 4] Train Loss: 0.4897, Val Loss: 0.4917,Val AUC: 0.7362072236952454, Test AUC: 0.7385

[Epoch 5] Train Loss: 0.4892, Val Loss: 0.4902,Val AUC: 0.736747135671258, Test AUC: 0.7394

[Epoch 6] Train Loss: 0.4887, Val Loss: 0.4894,Val AUC: 0.7375644401060377, Test AUC: 0.7393

[Epoch 7] Train Loss: 0.4882, Val Loss: 0.4911,Val AUC: 0.7365016297567042, Test AUC: 0.7391

[Epoch 8] Train Loss: 0.4878, Val Loss: 0.4914,Val AUC: 0.7366870916766378, Test AUC: 0.7395

[Epoch 9] Train Loss: 0.4876, Val Loss: 0.4914,Val AUC: 0.7363386851103552, Test AUC: 0.7395

[Epoch 10] Train Loss: 0.4872, Val Loss: 0.4926,Val AUC: 0.7371358

In [9]:
submission = pd.DataFrame({f'tabm_{data_seed}': np.mean(test_preds, axis=0)})
submission

,tabm_10
0,0.341478
1,0.105108
2,0.000232
3,0.114351
4,0.446093
...,...
51266,0.231313
51267,0.255745
51268,0.141795
51269,0.029850


In [10]:
submission.to_csv(f'TabM_{data_seed}.csv', index=False)

[seed 10]: 0.74150757679765
